In [1]:
import pyomo.environ as pyo
from pyomo.core.base.set import UnindexedComponent_set
#from farmer_example_block_robby import build_block_model
from pyomo.dae.flatten import flatten_components_along_sets
from pyomo.environ import *

In [2]:
def build_block_model(yields):
    '''
    Code adapted from https://mpi-sppy.readthedocs.io/en/latest/examples.html#examples
    It defines each scenario as a block 
    
    Arguments:
        yields: Yield information as a list, following the rank [wheat, corn, beets]
        
    Return: 
        model: farmer problem model 
    '''
    model = ConcreteModel()
    
    # Define sets
    model.all_crops = Set(initialize=["WHEAT", "CORN", "BEETS"])
    model.purchase_crops = Set(initialize=["WHEAT", "CORN"])
    model.sell_crops = Set(initialize=["WHEAT", "CORN", "BEETS_FAVORABLE", "BEETS_UNFAVORABLE"])
    
    # define scenario 
    model.scenarios = Set(initialize=['ABOVE','AVERAGE','BELOW'])
    
    # Crops field allocation
    model.X = Var(model.all_crops, within=NonNegativeReals)
    model.Y = Var(model.purchase_crops, within=NonNegativeReals)
    model.W = Var(model.sell_crops, within=NonNegativeReals)

    def construct_block(b, s):
        b.Y = Var(model.purchase_crops, within=NonNegativeReals)
        b.W = Var(model.sell_crops, within=NonNegativeReals)
    
    model.lsb = Block(model.scenarios, rule=construct_block)

    # Objective function
    model.PLANTING_COST = 150 * model.X["WHEAT"] + 230 * model.X["CORN"] + 260 * model.X["BEETS"]
    
    model.PURCHASE_COST = 238*sum(model.lsb[s].W["WHEAT"] for s in model.scenarios) + 210*sum(model.lsb[s].Y["CORN"] for s in model.scenarios)
    
    model.SALES_REVENUE_ABOVE = (
    170*model.lsb['ABOVE'].W["WHEAT"] + 150*model.lsb['ABOVE'].W["CORN"]
        + 36*model.lsb['ABOVE'].W["BEETS_FAVORABLE"] + 10*model.lsb['ABOVE'].W["BEETS_UNFAVORABLE"]
    )
    
    model.SALES_REVENUE_AVERAGE = (
    170*model.lsb['AVERAGE'].W["WHEAT"] + 150*model.lsb['AVERAGE'].W["CORN"]
        + 36*model.lsb['AVERAGE'].W["BEETS_FAVORABLE"] + 10*model.lsb['AVERAGE'].W["BEETS_UNFAVORABLE"]
    )
    
    model.SALES_REVENUE_BELOW = (
    170*model.lsb['BELOW'].W["WHEAT"] + 150*model.lsb['BELOW'].W["CORN"]
        + 36*model.lsb['BELOW'].W["BEETS_FAVORABLE"] + 10*model.lsb['BELOW'].W["BEETS_UNFAVORABLE"]
    )
    
    # Maximize the Obj is to minimize the negative of the Obj
    model.OBJ = Objective(
        expr=model.PLANTING_COST + 1/3*model.PURCHASE_COST - 1/3*(model.SALES_REVENUE_ABOVE + 
                                                                  model.SALES_REVENUE_AVERAGE+
                                                                  model.SALES_REVENUE_BELOW), sense=minimize)


    # Constraints
    model.CONSTR= ConstraintList()

    model.CONSTR.add(summation(model.X) <= 500)
    model.CONSTR.add(yields[0] * model.X["WHEAT"] + model.lsb['AVERAGE'].Y["WHEAT"] - model.lsb['AVERAGE'].W["WHEAT"] >= 200)
    model.CONSTR.add(yields[0]*1.2 * model.X["WHEAT"] + model.lsb['ABOVE'].Y["WHEAT"] - model.lsb['ABOVE'].W["WHEAT"] >= 200)
    model.CONSTR.add(yields[0]*0.8 * model.X["WHEAT"] + model.lsb['BELOW'].Y["WHEAT"] - model.lsb['BELOW'].W["WHEAT"] >= 200)
    
    model.CONSTR.add(yields[1] * model.X["CORN"] + model.lsb['AVERAGE'].Y["CORN"] - model.lsb['AVERAGE'].W["CORN"] >= 240)
    model.CONSTR.add(yields[1]*1.2 * model.X["CORN"] + model.lsb['ABOVE'].Y["CORN"] - model.lsb['ABOVE'].W["CORN"] >= 240)
    model.CONSTR.add(yields[1]*0.8 * model.X["CORN"] + model.lsb['BELOW'].Y["CORN"] - model.lsb['BELOW'].W["CORN"] >= 240)
    
    
    model.CONSTR.add(yields[2] * model.X["BEETS"] - model.lsb['AVERAGE'].W["BEETS_FAVORABLE"] - model.lsb['AVERAGE'].W["BEETS_UNFAVORABLE"] >= 0)
    model.CONSTR.add(yields[2]*1.2 * model.X["BEETS"] - model.lsb['ABOVE'].W["BEETS_FAVORABLE"] - model.lsb['ABOVE'].W["BEETS_UNFAVORABLE"] >= 0)
    model.CONSTR.add(yields[2]*0.8 * model.X["BEETS"] - model.lsb['BELOW'].W["BEETS_FAVORABLE"] - model.lsb['BELOW'].W["BEETS_UNFAVORABLE"] >= 0)
    
    model.lsb['AVERAGE'].W["BEETS_FAVORABLE"].setub(6000)
    model.lsb['ABOVE'].W["BEETS_FAVORABLE"].setub(6000)
    model.lsb['BELOW'].W["BEETS_FAVORABLE"].setub(6000)

    return model

In [3]:
def only_scenario_indexed():
    yields = [2.5, 3.0, 20.0]
    m = build_block_model(yields)

    sets = (m.scenarios,)
    # This partitions model components according to how they are indexed
    sets_list, vars_list = flatten_components_along_sets(
        m,
        sets,
        pyo.Var,
    )
    print('sets_list:', sets_list)
    print('vars_list:', vars_list)
    assert len(sets_list) <= 2
    assert len(vars_list) <= 2

    for sets, vars in zip(sets_list, vars_list):
        if len(sets) == 1 and sets[0] is UnindexedComponent_set:
            scalar_vars = vars
        elif len(sets) == 1 and sets[0] is m.scenarios:
            scenario_vars = vars
        else:
            # We only expect two cases here:
            # (a) unindexed
            # (b) indexed by scenario
            raise RuntimeError()

    sets_list, cons_list = flatten_components_along_sets(
        m,
        sets,
        pyo.Constraint,
    )
    assert len(sets_list) <= 2
    assert len(sets_list) <= 2
    scenario_cons = []
    for sets, cons in zip(sets_list, cons_list):
        if len(sets) == 1 and sets[0] is UnindexedComponent_set:
            scalar_cons = cons
        elif len(sets) == 1 and sets[0] is m.scenarios:
            scenario_cons = cons
        else:
            # We only expect two cases here:
            # (a) unindexed
            # (b) indexed by scenario
            raise RuntimeError()

    # The block hierarchy has been "flattened."
    # Not to be confused with "flattening" a high-dimension
    # index set into single-dimension index set.
    flattened_model = pyo.ConcreteModel()
    flattened_model.unindexed_vars = pyo.Reference(scalar_vars)
    for i, var in enumerate(scenario_vars):
        # var is already a reference.
        flattened_model.add_component("scenario_var_%s" % i, var)

    flattened_model.unindexed_constraints = pyo.Reference(scalar_cons)
    for i, con in enumerate(scenario_cons):
        flattened_model.add_component("scenario_con_%s" % i, con)

    flattened_model.obj = pyo.Reference(m.OBJ)

    solver = pyo.SolverFactory("ipopt")
    solver.solve(flattened_model, tee=True)
    
    return m, flattened_model


def all_sets():
    yields = [2.5, 3.0, 20.0]
    m = build_block_model(yields)

    # It makes sense to build the flattened model ahead of time
    # in this case, so we don't need to know what "set combinations"
    # we're looking for a priori
    flattened_model = pyo.ConcreteModel()

    sets = tuple(m.component_data_objects(pyo.Set))
    # This partitions model components according to how they are indexed
    sets_list, vars_list = flatten_components_along_sets(
        m,
        sets,
        pyo.Var,
    )
    for i, (sets, vars) in enumerate(zip(sets_list, vars_list)):
        print('i is:', i)
        print('sets is:', sets)
        print('vars is:', vars)
        if len(sets) == 1 and sets[0] is UnindexedComponent_set:
            flattened_model.unindexed_vars = pyo.Reference(vars)
        else:
            for j, var in enumerate(vars):
                flattened_model.add_component("var_%s_%s" % (i, j), var)

    sets_list, cons_list = flatten_components_along_sets(
        m,
        sets,
        pyo.Constraint,
    )
    for i, (sets, cons) in enumerate(zip(sets_list, cons_list)):
        if len(sets) == 1 and sets[0] is UnindexedComponent_set:
            flattened_model.unindexed_constraints = pyo.Reference(cons)
        else:
            for j, con in enumerate(cons):
                flattened_model.add_component("con_%s_%s" % (i, j), con)

    flattened_model.obj = pyo.Reference(m.OBJ)

    solver = pyo.SolverFactory("ipopt")
    solver.solve(flattened_model, tee=True)
    
    return m, flattened_model


model, flat1 = only_scenario_indexed()
model2, flat2 = all_sets()


sets_list: [(<pyomo.core.base.global_set._UnindexedComponent_set object at 0x177c05580>,), (<pyomo.core.base.set.OrderedScalarSet object at 0x178280120>,)]
vars_list: [[<pyomo.core.base.var._GeneralVarData object at 0x17827f3a0>, <pyomo.core.base.var._GeneralVarData object at 0x17827f640>, <pyomo.core.base.var._GeneralVarData object at 0x17827f6a0>, <pyomo.core.base.var._GeneralVarData object at 0x17827f700>, <pyomo.core.base.var._GeneralVarData object at 0x17827f760>, <pyomo.core.base.var._GeneralVarData object at 0x17827f7c0>, <pyomo.core.base.var._GeneralVarData object at 0x17827f820>, <pyomo.core.base.var._GeneralVarData object at 0x17827f880>, <pyomo.core.base.var._GeneralVarData object at 0x17827f8e0>], [<pyomo.core.base.var.IndexedVar object at 0x17829db80>, <pyomo.core.base.var.IndexedVar object at 0x17828dfa0>, <pyomo.core.base.var.IndexedVar object at 0x178274880>, <pyomo.core.base.var.IndexedVar object at 0x178274bb0>, <pyomo.core.base.var.IndexedVar object at 0x1782747c0>, 

In [11]:
#print(value(flat1.scenario_var_0["ABOVE"]))
print(value(flat2.var_3_0["ABOVE","CORN"]))

for i in flat2.var_4_0_index:
    print(i)

0.0
('ABOVE', 'WHEAT')
('ABOVE', 'CORN')
('ABOVE', 'BEETS_FAVORABLE')
('ABOVE', 'BEETS_UNFAVORABLE')
('AVERAGE', 'WHEAT')
('AVERAGE', 'CORN')
('AVERAGE', 'BEETS_FAVORABLE')
('AVERAGE', 'BEETS_UNFAVORABLE')
('BELOW', 'WHEAT')
('BELOW', 'CORN')
('BELOW', 'BEETS_FAVORABLE')
('BELOW', 'BEETS_UNFAVORABLE')


In [9]:
print(value(flat2.Y["CORN"]))

AttributeError: 'ConcreteModel' object has no attribute 'Y'

In [10]:
#print(value(flat1.lsb["ABOVE"].W["CORN"]))
#print(value(flat1.W["CORN","ABOVE"]))
display (flat2)

Model unknown

  Variables:
    var_0_0 : Size=3, Index=all_crops
        Key   : Lower : Value              : Upper : Fixed : Stale : Domain
        BEETS :     0 : 300.00000299895424 :  None : False : False : NonNegativeReals
         CORN :     0 :  200.0000020110286 :  None : False : False : NonNegativeReals
        WHEAT :     0 :                0.0 :  None : False : False : NonNegativeReals
    var_1_0 : Size=2, Index=purchase_crops
        Key   : Lower : Value : Upper : Fixed : Stale : Domain
         CORN :     0 :  None :  None : False :  True : NonNegativeReals
        WHEAT :     0 :  None :  None : False :  True : NonNegativeReals
    var_2_0 : Size=4, Index=sell_crops
        Key               : Lower : Value : Upper : Fixed : Stale : Domain
          BEETS_FAVORABLE :     0 :  None :  None : False :  True : NonNegativeReals
        BEETS_UNFAVORABLE :     0 :  None :  None : False :  True : NonNegativeReals
                     CORN :     0 :  None :  None : False :  Tru

In [ ]:
display(model)

In [12]:
yields_perfect = [2.5, 3, 20]

print("===Optimal solutions of two-stage stochastic problem with blocks===")
print('Culture.         | ', 'Wheat |', 'Corn  |', 'Sugar Beets |')
print('Surface (acres)  | ', f'{value(model.X["WHEAT"]):.1f}', '|', 
      f'{value(model.X["CORN"]):.1f}', ' |',
       f'{value(model.X["BEETS"]):.1f}',' |')
print('First stage: s=1 (Above average)')
print('Culture.         | ', 'Wheat |', 'Corn  |', 'Sugar Beets |')
print('Yield (T)        | ', f'{value(model.X["WHEAT"])*yields_perfect[0]*1.2:.1f}', '|', 
      f'{value(model.X["CORN"])*yields_perfect[1]*1.2:.1f}', '|',
       f'{value(model.X["BEETS"])*yields_perfect[2]*1.2:.1f}','|')

print('Sales (T)        | ', f'{value(model.lsb["ABOVE"].W["WHEAT"]):.1f}', '|', 
      f'{value(model.lsb["ABOVE"].W["CORN"]):.1f}', '|',
       f'{value(model.lsb["ABOVE"].W["BEETS_FAVORABLE"]) + value(model.lsb["ABOVE"].W["BEETS_UNFAVORABLE"]):.1f}','|')
print('Purchases (T)    | ', f'{value(model.lsb["ABOVE"].Y["WHEAT"]):.1f}', '|', 
      f'{value(model.lsb["ABOVE"].Y["CORN"]):.1f}', '  |',
       '-','     |')

print('First stage: s=2 (Average average)')
print('Culture.         | ', 'Wheat |', 'Corn  |', 'Sugar Beets |')
print('Yield (T)        | ', f'{value(model.X["WHEAT"])*yields_perfect[0]:.1f}', '|', 
      f'{value(model.X["CORN"])*yields_perfect[1]:.1f}', '|',
       f'{value(model.X["BEETS"])*yields_perfect[2]:.1f}','|')
print('Sales (T)        | ', f'{value(model.lsb["AVERAGE"].W["WHEAT"]):.1f}', '|', 
      f'{value(model.lsb["AVERAGE"].W["CORN"]):.1f}', '  |',
       f'{value(model.lsb["AVERAGE"].W["BEETS_FAVORABLE"]) + value(model.lsb["AVERAGE"].W["BEETS_UNFAVORABLE"]):.1f}','|')
print('Purchases (T)    | ', f'{value(model.lsb["AVERAGE"].Y["WHEAT"]):.1f}', '  |', 
      f'{value(model.lsb["AVERAGE"].Y["CORN"]):.1f}', '  |',
       '-','     |')

print('First stage: s=3 (Below average)')
print('Culture.         | ', 'Wheat |', 'Corn  |', 'Sugar Beets |')
print('Yield (T)        | ', f'{value(model.X["WHEAT"])*yields_perfect[0]*0.8:.1f}', '|', 
      f'{value(model.X["CORN"])*yields_perfect[1]*0.8:.1f}', '|',
       f'{value(model.X["BEETS"])*yields_perfect[2]*0.8:.1f}','|')
print('Sales (T)        | ', f'{value(model.lsb["BELOW"].W["WHEAT"]):.1f}', '|', 
      f'{value(model.lsb["BELOW"].W["CORN"]):.1f}', '  |',
       f'{value(model.lsb["BELOW"].W["BEETS_FAVORABLE"]) + value(model.lsb["BELOW"].W["BEETS_UNFAVORABLE"]):.1f}','|')
print('Purchases (T)    | ', f'{value(model.lsb["BELOW"].Y["WHEAT"]):.1f}', '  |', 
      f'{value(model.lsb["BELOW"].Y["CORN"]):.1f}', '  |',
       '-','     |')

===Optimal solutions of two-stage stochastic problem with blocks===
Culture.         |  Wheat | Corn  | Sugar Beets |
Surface (acres)  |  0.0 | 200.0  | 300.0  |
First stage: s=1 (Above average)
Culture.         |  Wheat | Corn  | Sugar Beets |
Yield (T)        |  0.0 | 720.0 | 7200.0 |
Sales (T)        |  0.0 | 480.0 | 7200.0 |
Purchases (T)    |  1646.5 | 0.0   | -      |
First stage: s=2 (Average average)
Culture.         |  Wheat | Corn  | Sugar Beets |
Yield (T)        |  0.0 | 600.0 | 6000.0 |
Sales (T)        |  0.0 | 360.0   | 6000.0 |
Purchases (T)    |  1646.5   | 0.0   | -      |
First stage: s=3 (Below average)
Culture.         |  Wheat | Corn  | Sugar Beets |
Yield (T)        |  0.0 | 480.0 | 4800.0 |
Sales (T)        |  0.0 | 240.0   | 4800.0 |
Purchases (T)    |  1646.5   | 0.0   | -      |
